# 🌊 GSR/GPS Exploration Dashboard

Schöneres, interaktives UI für:

- Auswahl eines **Runs** (CSV aus `results/csv`)
- Interaktive **GSR-Zeitreihen-Ansicht** (Fenster-Slider)
- **SCR-Statistik** (Histogramm + Kennzahlen)
- **GPS-Ansichten** (Heatmap & Punkte, filterbar nach Peaks/Triggern)

> Tipp: Notebook im **Browser** (klassisches Jupyter Notebook) öffnen, nicht in VS Code.

In [6]:
import sys
from pathlib import Path

project_root = Path('..').resolve()
sys.path.append(str(project_root))
project_root

WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer')

In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Größere, besser sichtbare Widgets
HTML('''
<style>
.widget-label { 
    font-size: 12pt !important;
}
.widget-dropdown, .widget-text, .widget-slider {
    font-size: 12pt !important;
}
.widget-slider .noUi-target {
    height: 10px !important;
}
.widget-slider .noUi-handle {
    width: 18px !important;
    height: 18px !important;
}
</style>
''')

## 1. Run auswählen

Hier werden alle Ergebnistabellen aus `results/csv` gelistet. Mit dem Dropdown wählst du den Run, mit dem du arbeiten möchtest.


In [8]:
csv_dir = project_root / 'results' / 'csv'
csv_files = sorted(csv_dir.glob('output_GSR_GPS_*.csv')) if csv_dir.exists() else []

if not csv_files:
    print('⚠️ Keine CSV-Dateien in results/csv gefunden. Bitte zuerst die Pipeline ausführen.')
csv_files

[WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS_Neckarhalde_run12_female_5031975_20251125_131507.csv'),
 WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS_Neckarhalde_run13_female_5031975_20251125_114222.csv'),
 WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS_Neckarhalde_run13_female_5031975_20251125_115608.csv'),
 WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS_Neckarhalde_run13_female_5031975_20251125_120647.csv'),
 WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS_Neckarhalde_run13_female_5031975_20251125_122340.csv'),
 WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS_Neckarhalde_run13_female_5031975_20251125_131422.csv'),
 WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer/results/csv/output_GSR_GPS

In [9]:
current_df = None
current_file = None

run_box = widgets.VBox()

if csv_files:
    run_dropdown = widgets.Dropdown(
        options=[(p.name, i) for i, p in enumerate(csv_files)],
        description='Run:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='450px')
    )
    reload_button = widgets.Button(description='Daten laden', button_style='primary', icon='refresh')
    info_out = widgets.Output()

    def load_selected_df(*args):
        global current_df, current_file
        idx = run_dropdown.value
        path = csv_files[idx]
        df = pd.read_csv(path)
        if 'Timestamp' in df.columns:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        current_df = df
        current_file = path
        with info_out:
            clear_output()
            display(HTML(f'<b>Aktive Datei:</b> {path.name}'))
            display(df.head())

    reload_button.on_click(load_selected_df)
    load_selected_df()

    run_box.children = [widgets.HBox([run_dropdown, reload_button]), info_out]
    display(run_box)
else:
    display(run_box)


## 2. Dashboard-Tabs

Unten findest du drei Tabs:
- **GSR** – Zeitreihe + Fenster-Slider
- **SCR-Stats** – Histogramm + Kennzahlen
- **GPS** – Heatmap & punktweise Ansicht (alle/Peaks/Trigger)


In [10]:
def ensure_df():
    if current_df is None:
        print('⚠️ Noch keine Daten geladen. Bitte oben einen Run wählen und "Daten laden" drücken.')
        return None
    return current_df

# ---------- GSR Tab ----------
gsr_out = widgets.Output(layout=widgets.Layout(border='1px solid #ccc'))

def gsr_plot_window(start_idx, window_size):
    df = ensure_df()
    if df is None:
        return
    if 'Timestamp' not in df.columns or 'Conductance' not in df.columns:
        with gsr_out:
            clear_output()
            print('Spalten Timestamp/Conductance fehlen.')
        return

    n = len(df)
    start_idx = max(0, min(start_idx, n-1))
    end_idx = max(0, min(start_idx + window_size, n))
    d = df.iloc[start_idx:end_idx].copy()
    if d.empty:
        with gsr_out:
            clear_output()
            print('Zeitfenster leer.')
        return

    with gsr_out:
        clear_output()
        fig, ax = plt.subplots(figsize=(12,4))
        ax.plot(d['Timestamp'], d['Conductance'], label='Conductance', alpha=0.8)
        if 'SCL_Global' in d.columns:
            ax.plot(d['Timestamp'], d['SCL_Global'], label='SCL Global', linewidth=1.5)
        if 'SCL_Baseline' in d.columns:
            ax.plot(d['Timestamp'], d['SCL_Baseline'], label='SCL Baseline', linewidth=1.5)
        if 'Trigger' in d.columns:
            trigs = d[d['Trigger'] == 1]
            if not trigs.empty:
                ax.scatter(trigs['Timestamp'], trigs['Conductance'], color='red', marker='*', s=80, label='Trigger')
        if 'SCR_Peak' in d.columns:
            peaks = d[d['SCR_Peak'] == 1]
            if not peaks.empty:
                ax.scatter(peaks['Timestamp'], peaks['Conductance'], color='orange', marker='^', s=60, label='SCR Peak')
        ax.set_xlabel('Zeit')
        ax.set_ylabel('Conductance (µS)')
        ax.legend()
        ax.grid(alpha=0.3)
        fig.autofmt_xdate()
        plt.tight_layout()
        plt.show()

def make_gsr_tab():
    df = ensure_df()
    if df is None:
        return widgets.VBox([widgets.HTML('<b>Keine Daten geladen.</b>')])

    n = len(df)
    start_slider = widgets.IntSlider(
        min=0, max=max(n-1, 1), step=max(1, n//100), value=0,
        description='Start-Index:',
        layout=widgets.Layout(width='600px')
    )
    window_slider = widgets.IntSlider(
        min=60, max=max(600, n), step=60, value=min(600, n),
        description='Fenstergröße:',
        layout=widgets.Layout(width='600px')
    )

    ui = widgets.VBox([
        widgets.HTML('<b>GSR-Zeitfenster</b>'),
        start_slider,
        window_slider,
    ])
    out = widgets.interactive_output(gsr_plot_window, {
        'start_idx': start_slider,
        'window_size': window_slider
    })
    return widgets.VBox([ui, gsr_out])

# ---------- SCR Stats Tab ----------
scr_out = widgets.Output(layout=widgets.Layout(border='1px solid #ccc'))

def update_scr_stats():
    df = ensure_df()
    with scr_out:
        clear_output()
        if df is None:
            return
        if 'SCR_Latency_s' not in df.columns:
            print('Keine SCR_Latency_s-Spalte vorhanden.')
            return
        lat = df['SCR_Latency_s'].dropna()
        if lat.empty:
            print('Keine SCR-Latenzen vorhanden.')
            return
        fig, ax = plt.subplots(figsize=(8,4))
        ax.hist(lat, bins=np.arange(0, 10, 0.5), edgecolor='k')
        ax.set_xlabel('Latenz (s)')
        ax.set_ylabel('Anzahl')
        ax.set_title('Histogramm der SCR-Latenzen')
        ax.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        display(pd.DataFrame({
            'Anzahl Peaks': [len(lat)],
            'Mittelwert (s)': [lat.mean()],
            'Median (s)': [lat.median()],
            'Std (s)': [lat.std()]
        }))

def make_scr_tab():
    btn = widgets.Button(description='SCR-Statistik aktualisieren', icon='refresh')
    btn.on_click(lambda _: update_scr_stats())
    update_scr_stats()
    return widgets.VBox([
        widgets.HTML('<b>SCR-Statistik</b>'),
        btn,
        scr_out
    ])

# ---------- GPS Tab ----------
gps_out = widgets.Output(layout=widgets.Layout(border='1px solid #ccc'))

def plot_gps(mode='all', as_hexbin=False):
    df = ensure_df()
    with gps_out:
        clear_output()
        if df is None:
            return
        if not {'latitude', 'longitude'}.issubset(df.columns):
            print('Spalten latitude/longitude fehlen.')
            return
        d = df.dropna(subset=['latitude', 'longitude'])
        if d.empty:
            print('Keine GPS-Daten vorhanden.')
            return
        if mode == 'peaks':
            d = d[d.get('SCR_Peak', 0) == 1]
            title = 'GPS: SCR-Peaks'
        elif mode == 'triggers':
            d = d[d.get('Trigger', 0) == 1]
            title = 'GPS: Trigger'
        else:
            title = 'GPS: Alle Punkte'

        if d.empty:
            print('Keine Daten für diesen Modus.')
            return

        fig, ax = plt.subplots(figsize=(6,6))
        if as_hexbin:
            hb = ax.hexbin(d['longitude'], d['latitude'], gridsize=40, cmap='viridis', mincnt=1)
            cb = fig.colorbar(hb, ax=ax)
            cb.set_label('Anzahl Punkte')
        else:
            ax.scatter(d['longitude'], d['latitude'], s=10, alpha=0.7)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_title(title)
        plt.tight_layout()
        plt.show()

def make_gps_tab():
    mode_dropdown = widgets.Dropdown(
        options=[('Alle Punkte', 'all'), ('Nur Peaks', 'peaks'), ('Nur Trigger', 'triggers')],
        value='all',
        description='Modus:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    hex_toggle = widgets.ToggleButtons(
        options=[('Punkte', False), ('Heatmap (Hexbin)', True)],
        value=False,
        description='Darstellung:',
        style={'description_width': 'initial'}
    )

    ui = widgets.VBox([
        widgets.HTML('<b>GPS-Ansicht</b>'),
        widgets.HBox([mode_dropdown, hex_toggle])
    ])
    out = widgets.interactive_output(plot_gps, {
        'mode': mode_dropdown,
        'as_hexbin': hex_toggle
    })
    return widgets.VBox([ui, gps_out])

# ---------- Tabs zusammensetzen ----------
gsr_tab = make_gsr_tab()
scr_tab = make_scr_tab()
gps_tab = make_gps_tab()

tabs = widgets.Tab(children=[gsr_tab, scr_tab, gps_tab])
tabs.set_title(0, 'GSR')
tabs.set_title(1, 'SCR-Stats')
tabs.set_title(2, 'GPS')
display(tabs)